In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp

import tensorflow.keras as tf_keras
from sklearn.preprocessing import StandardScaler

import mlflow
import mlflow.sklearn
import mlflow.keras

import importlib_metadata

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils import class_weight



2022-03-11 07:17:03.326371: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 07:17:03.326415: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.6.0'

In [3]:
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

In [4]:
df_all = pd.read_csv('hf_model_seg2.csv',sep=',', encoding='cp949')
df_dev = df_all[df_all.val_gb == 0]
df_val = df_all[df_all.val_gb == 1]

# h2o_dtrain['target_6m'] = h2o_dtrain['target_6m'].asfactor()
# h2o_dvalid['target_6m'] = h2o_dvalid['target_6m'].asfactor()

selected_var_card = ['AE0000012','AE0000016','AS0000136','AS0000137','AS0000144','C00000077','C00000080','C00000082','C00000085','C00000090','C00000093',
'C00060606','CA0000101','CA0000603','CA0000803','CA0300002','CA0600002','CA1200002','CF0100236','CF0300904','CF1200622','CF1200917',
'CF1231601','CS0000011','CS0000025','CS0000050','CS0000202','GRD_PD0801_000','IDT000003','IE0300005','IE0300012','KC0300001','KC1000033',
'L03009901','L03089901','L06009901','L12000004','L12000701','L12009901','L21213700','L22000900','L22001100','L22001300','L23001002',
'L23001911','L23001913','L29001205','LA0000261','LA0000601','LA0300107','LA0308206','LA0508201','LA0608206','LA1200106','LC0000010',
'LC0000109','LC0000203','LC0000208','LC0000210','LC0000233','LC0000601','LC0000609','LC0025010','LC0099901','LC0321202','LC0600204',
'LC1200103','LC1200201','LF3600201','LF9900012','LH0000001','LH0000002','LH0000003','LH0000153','LH0000601','LHC000005','LRZ021208',
'LRZ021210','LRZ021213','LS0000162','PH0000225','PHC000007','PS0001252','RABAC0001','RABAC0016','SC0000059','SC0000060','SC0000090',
'SC0601005','SC0602005','SC1201005','SC3601005','SC3602005','SCR_PD2400_000','SCR_RK0100_201','SCR_RK0200_001','SCR_RK0600_000','SCR_RK0600_700','ZS0000182','ZS0000183']



# dev_df = pd.read_csv('ML_data_dev.txt',sep='|' )
# val_df = pd.read_csv('ML_data_val.txt',sep='|' )

In [5]:
dev_df_y = pd.get_dummies(df_dev.target_6m) 
val_df_y = pd.get_dummies(df_val.target_6m)
dev_df_x = df_dev[selected_var_card]
val_df_x = df_val[selected_var_card]

In [6]:
x_train = dev_df_x.values
y_train = dev_df_y.values
x_val = val_df_x.values
y_val = val_df_y.values

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

x_train

array([[ 2.73054055,  0.66781837,  1.86289819, ...,  0.83724373,
         1.33109892,  0.7922558 ],
       [-0.51663258,  1.10489348, -1.23049091, ..., -0.10099549,
         0.65143723,  0.26878659],
       [ 2.6655217 ,  0.23074326, -1.23049091, ...,  0.4306734 ,
        -0.70788615,  0.26878659],
       ...,
       [-0.51286337,  0.23074326, -1.23049091, ...,  0.24302556,
        -0.02822446,  0.26878659],
       [-0.72205445,  0.66781837, -0.09624824, ..., -0.94541079,
        -0.70788615, -1.30162107],
       [-0.8351307 , -0.64340696, -0.09624824, ..., -0.38246726,
        -0.02822446, -0.77815185]])

In [8]:
def get_initializer(activation_func):
    if activation_func == 'sigmoid' or activation_func == 'tanh':
        return 'glorot_normal'
    elif activation_func == 'relu' or activation_func == 'elu' or activation_func == 'selu':
        return 'he_normal'
    else:
        return 'glorot_normal'
    
def get_optimizer(optimizer_func_name, learning_rate):
    opt = None
    if optimizer_func_name == 'SGD':
        opt = tf_keras.optimizers.SGD(lr=learning_rate)
    elif optimizer_func_name == 'RMSprop':
        opt = tf_keras.optimizers.RMSprop(lr=learning_rate)
    elif optimizer_func_name == 'Adagrad':
        opt = tf_keras.optimizers.Adagrad(lr=learning_rate)
    elif optimizer_func_name == 'Adadelta':
        opt = tf_keras.optimizers.Adadelta(lr=learning_rate)
    elif optimizer_func_name == 'Adam':
        opt = tf_keras.optimizers.Adam(lr=learning_rate)
    elif optimizer_func_name == 'Adamax':
        opt = tf_keras.optimizers.Adamax(lr=learning_rate)
    else:
        opt = tf_keras.optimizers.Adam(lr=learning_rate)
    return opt

# Get output layer's activation function according to the type of the target
def get_output_activation_func(is_target_discrete):
    return 'softmax' if is_target_discrete else 'linear'

In [9]:
activation_func = 'relu'
l1_penalty = 0.00001
l2_penalty = 0.00001
input_dropout_ratio = 0.1
is_target_discrete = True
hidden_layers_info = [50, 50, 50]
OutputDropoutRatio = [0.15, 0.15, 0.15]
kernel_regularizer = tf_keras.regularizers.l1_l2(l1=l1_penalty, l2=l2_penalty)
kernel_initializer = get_initializer(activation_func)
output_activation = get_output_activation_func(is_target_discrete)
num_input_nodes = len(selected_var_card)
use_batch_normalization = True
num_output_layers = 2

model = tf_keras.models.Sequential()

if len(hidden_layers_info) > 0:
    # input dropout ratio
    if input_dropout_ratio > 0.0:
        model.add(tf_keras.layers.Dropout(input_dropout_ratio, input_shape=(num_input_nodes,),
                            name='input_layer'))
    ## Hidden layers
    for i in range(0, len(hidden_layers_info)):
        if (i == 0 and input_dropout_ratio > 0.0) or (i > 0):
            model.add(tf_keras.layers.Dense(units=hidden_layers_info[i],
                            kernel_regularizer=kernel_regularizer,
                            kernel_initializer=kernel_initializer,
                            name='hidden' + str(i)))
        else:
            model.add(tf_keras.layers.Dense(units=hidden_layers_info[i],
                            input_shape=(num_input_nodes,),
                            kernel_regularizer=kernel_regularizer,
                            kernel_initializer=kernel_initializer,
                            name='hidden' + str(i)))
        # [2017-03-30] pjh - Batch normalization added
        if use_batch_normalization:
            model.add(tf_keras.layers.BatchNormalization())
        model.add(tf_keras.layers.Activation(activation_func))
        # [2017-03-29] pjh - Keras's DropOut works for the ratio greater than 0.0
        if OutputDropoutRatio[i] > 0.0:
            model.add(tf_keras.layers.Dropout(OutputDropoutRatio[i]))

    model.add(tf_keras.layers.Dense(units=num_output_layers, name='output_layer', activation=output_activation))
else: # no hidden layer
    model.add(tf_keras.layers.Dense(units=num_output_layers,  input_shape=(num_input_nodes, ), activation=output_activation, name='output_layer'))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy'])

# model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(0.001),
#             loss=tf.keras.losses.BinaryCrossentropy(),
#             metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dropout)        (None, 100)               0         
_________________________________________________________________
hidden0 (Dense)              (None, 50)                5050      
_________________________________________________________________
batch_normalization (BatchNo (None, 50)                200       
_________________________________________________________________
activation (Activation)      (None, 50)                0         
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
hidden1 (Dense)              (None, 50)                2550      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50)                2

2022-03-11 07:20:06.926930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-11 07:20:06.926977: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-11 07:20:06.926996: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-30-7-116): /proc/driver/nvidia/version does not exist
2022-03-11 07:20:06.927218: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# mlflow.tensorflow.autolog(log_models=True)

from sklearn.utils import class_weight

mlflow.set_tracking_uri("http://localhost:3000")
# mlflow.log_metric("binary_loss", tf.keras.losses.BinaryCrossentropy())

class_weights = dict(zip(np.unique(y_train[:,0]), class_weight.compute_class_weight(class_weight = 'balanced', classes=np.unique(y_train[:,0]), 
                y =y_train[:,0]))) 
n_epochs = 5
batch_size = 1024
has_validation_set = True

with mlflow.start_run() as run:
    mlflow.tensorflow.autolog()
    result = model.fit(x_train, y_train,
                    epochs=n_epochs, batch_size=batch_size, class_weight= class_weights,
                    validation_data=(x_val, y_val) if has_validation_set else None, verbose=2)
    
    # mlflow.keras.log_model(model, "keras_seg2_model_non_ewc", keras_module=tf_keras)
    # mlflow.keras.log_model(model, "keras_seg2_model_non_ewc")


2022-03-11 08:34:03.244276: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-03-11 08:34:03.244326: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-03-11 08:34:03.244434: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/5


2022-03-11 08:34:03.572039: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-03-11 08:34:03.572092: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-03-11 08:34:03.611534: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-03-11 08:34:03.613221: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-03-11 08:34:03.615743: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /tmp/tmp1zklzyqx/train/plugins/profile/2022_03_11_08_34_03

2022-03-11 08:34:03.617111: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /tmp/tmp1zklzyqx/train/plugins/profile/2022_03_11_08_34_03/ip-172-30-7-116.trace.json.gz
2022-03-11 08:34:03.619444: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /tmp/tmp1zklzyqx/train/plugins/profile/2022_03_11_

302/302 - 3s - loss: 0.2788 - accuracy: 0.8939 - val_loss: 0.3926 - val_accuracy: 0.8894
Epoch 2/5
302/302 - 3s - loss: 0.2772 - accuracy: 0.8939 - val_loss: 0.3976 - val_accuracy: 0.8894
Epoch 3/5
302/302 - 3s - loss: 0.2765 - accuracy: 0.8939 - val_loss: 0.4053 - val_accuracy: 0.8894
Epoch 4/5
302/302 - 3s - loss: 0.2748 - accuracy: 0.8939 - val_loss: 0.4058 - val_accuracy: 0.8894
Epoch 5/5
302/302 - 3s - loss: 0.2741 - accuracy: 0.8939 - val_loss: 0.3938 - val_accuracy: 0.8894
INFO:tensorflow:Assets written to: /tmp/tmphnwdnz2s/model/data/model/assets


In [19]:
pred_dev = model.predict(x_train)
pred_val = model.predict(x_val)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
# reduce gap between train KS and test KS
ks_dev = ks_stat(df_dev.target_6m, pred_dev[:,1])
ks_val = ks_stat(df_val.target_6m, pred_val[:,1])

print(ks_dev)
print(ks_val)

0.5455742628193866
0.5139843158299896


In [20]:
# validation으로 추가 학습
n_epochs = 100
batch_size = 1024
result = model.fit(x_val, y_val,
                        epochs=n_epochs, batch_size=batch_size, class_weight= class_weights,
                        validation_split= 0.3, verbose=2)

Epoch 1/100
29/29 - 0s - loss: 0.2832 - accuracy: 0.8901 - val_loss: 0.3856 - val_accuracy: 0.8878
Epoch 2/100
29/29 - 0s - loss: 0.2788 - accuracy: 0.8901 - val_loss: 0.3989 - val_accuracy: 0.8878
Epoch 3/100
29/29 - 0s - loss: 0.2773 - accuracy: 0.8901 - val_loss: 0.3998 - val_accuracy: 0.8878
Epoch 4/100
29/29 - 0s - loss: 0.2769 - accuracy: 0.8901 - val_loss: 0.4050 - val_accuracy: 0.8878
Epoch 5/100
29/29 - 0s - loss: 0.2758 - accuracy: 0.8901 - val_loss: 0.4092 - val_accuracy: 0.8878
Epoch 6/100
29/29 - 0s - loss: 0.2745 - accuracy: 0.8902 - val_loss: 0.4160 - val_accuracy: 0.8878
Epoch 7/100
29/29 - 0s - loss: 0.2745 - accuracy: 0.8903 - val_loss: 0.4127 - val_accuracy: 0.8878
Epoch 8/100
29/29 - 0s - loss: 0.2721 - accuracy: 0.8903 - val_loss: 0.4151 - val_accuracy: 0.8878
Epoch 9/100
29/29 - 0s - loss: 0.2726 - accuracy: 0.8901 - val_loss: 0.4188 - val_accuracy: 0.8878
Epoch 10/100
29/29 - 0s - loss: 0.2737 - accuracy: 0.8901 - val_loss: 0.4196 - val_accuracy: 0.8878
Epoch 11/

In [21]:
pred_dev = model.predict(x_train)
pred_val = model.predict(x_val)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
# reduce gap between train KS and test KS
ks_dev = ks_stat(df_dev.target_6m, pred_dev[:,1])
ks_val = ks_stat(df_val.target_6m, pred_val[:,1])

print(ks_dev)
print(ks_val)

0.44897238907050063
0.6224881544104656


In [22]:
def prepare(ds, shuffle=True, batch_size=32, prefetch=True):
    ds = ds.shuffle(ds_info.splits['train'].num_examples) if shuffle else ds
    ds = ds.cache()
    ds = ds.batch(batch_size)
    return ds

def evaluate(model, test_set):
  # acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
  acc = tf.keras.metrics.BinaryAccuracy(name='accuracy')
  for i, (imgs, labels) in enumerate(test_set):
    preds = model.predict_on_batch(imgs)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [23]:
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
  task_set = task_set.repeat()
  precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

  for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
    # We need gradients of model params
    with tf.GradientTape() as tape:
      # Get model predictions for each image
      preds = model(imgs)
      # Get the log likelihoods of the predictions
      ll = tf.nn.log_softmax(preds)
    # Attach gradients of ll to ll_grads
    ll_grads = tape.gradient(ll, model.trainable_variables)
    # Compute F_i as mean of gradients squared
    for i, g in enumerate(ll_grads):
      precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

  return precision_matrices

def compute_elastic_penalty(F, theta, theta_A, alpha=1):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*alpha*penalty

def ewc_loss(labels, preds, model, F, theta_A):
  loss_b = model.loss(labels, preds)
  penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
  return loss_b + penalty

def train_with_ewc(model, task_A_set, task_B_set, epochs=3):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  # model.fit(task_A_set, epochs=epochs, steps_per_epoch=1)
  # model.fit(task_A_set, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
  # We'll only compute Fisher once, you can do it whenever
  F = compute_precision_matrices(model, task_A_set, num_batches=1000)

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_set)))
  print("Task B accuracy after training on Task A: {}".format(evaluate(model, task_B_set))) 

  # Now we set up the training loop for task B with EWC
  accuracy = tf.keras.metrics.BinaryAccuracy('accuracy')
  loss = tf.keras.metrics.BinaryCrossentropy('loss')

  # for epoch in range(epochs*3):
  for epoch in range(epochs):
    accuracy.reset_states()
    loss.reset_states()

    for batch, (imgs, labels) in enumerate(task_B_set):
      with tf.GradientTape() as tape:
        # Make the predictions
        preds = model(imgs)
        # Compute EWC loss
        total_loss = ewc_loss(labels, preds, model, F, theta_A)
      # Compute the gradients of model's trainable parameters wrt total loss
      grads = tape.gradient(total_loss, model.trainable_variables)
      # Update the model with gradients
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # Report updated loss and accuracy
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_set)))
  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_set)))


In [25]:
task_A_set = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train.astype(np.float32))).shuffle(len(x_train)).batch(1024)
task_B_set = tf.data.Dataset.from_tensor_slices((x_val.astype(np.float32), y_val.astype(np.float32))).shuffle(len(x_val)).batch(1024)

model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(0.001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy'])
model.fit(task_A_set, epochs=100)

pred_dev = model.predict(x_train)
pred_val = model.predict(x_val)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
# reduce gap between train KS and test KS
ks_dev = ks_stat(df_dev.target_6m, pred_dev[:,1])
ks_val = ks_stat(df_val.target_6m, pred_val[:,1])

print(ks_dev)
print(ks_val)

train_with_ewc(model, task_A_set, task_B_set, epochs=100)

pred_dev = model.predict(x_train)
pred_val = model.predict(x_val)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
# reduce gap between train KS and test KS
ks_dev = ks_stat(df_dev.target_6m, pred_dev[:,1])
ks_val = ks_stat(df_val.target_6m, pred_val[:,1])

print(ks_dev)
print(ks_val)

Epoch 1/100
302/302 [==============================] - 6s 13ms/step - loss: 0.2630 - accuracy: 0.9019
Epoch 2/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2626 - accuracy: 0.9018
Epoch 3/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2624 - accuracy: 0.9017
Epoch 4/100
302/302 [==============================] - 4s 12ms/step - loss: 0.2616 - accuracy: 0.9024
Epoch 5/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2620 - accuracy: 0.9020
Epoch 6/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2609 - accuracy: 0.9023
Epoch 7/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2614 - accuracy: 0.9017
Epoch 8/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2610 - accuracy: 0.9021
Epoch 9/100
302/302 [==============================] - 4s 13ms/step - loss: 0.2605 - accuracy: 0.9020
Epoch 10/100
302/302 [==============================] - 4s 13ms/step - loss: 0.260

2022-03-10 08:36:07.734191: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled
2022-03-10 08:36:07.750132: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


Task B accuracy after training on Task A: 0.8978545665740967
Epoch: 1, Batch: 41, Loss: 0.263, Accuracy: 0.899
Epoch: 2, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 3, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 4, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 5, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 6, Batch: 41, Loss: 0.262, Accuracy: 0.900
Epoch: 7, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 8, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 9, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 10, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 11, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 12, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 13, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 14, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 15, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 16, Batch: 41, Loss: 0.262, Accuracy: 0.900
Epoch: 17, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 18, Batch: 41, Loss: 0.262, Accuracy: 0.899
Epoch: 19, Batch: 41, Loss: 0.

2022-03-10 08:39:14.002123: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


Task A accuracy after training trained model on Task B: 0.9064446091651917
Task B accuracy after training trained model on Task B: 0.8999046683311462
0.5732332778624253
0.5252725255009326


In [24]:
# save fitted model
model.save_weights("current_weight.h5")
with open('currnet_arch.json', 'w') as arch_json_fd:
    arch_json_fd.write(model.to_json())

# model은 정확하게 migration안될 수 있으나, weight는 확실히 migration됨

In [23]:
# 1년 정도 모형을 만든 후, 3개월 지난 후 부터 학습을 시작함
# 실행건에 대하여 학습 수행, 2020년 6월까지 모형을 만들고, 7월부터 '21.3월까지 모형의 Performance를 확인함(9개월)
# 10월 부터 7월 데이터를 활용하여 재적합하고, 모형의 Performance를 체크함
date_cl_nl = "appl_dt"
dt = 202001
while dt <= 202103:
    print(dt)
    # write logic in this part
    if str(dt)[4:6]=="12":
        dt += 89
    else: dt += 1



202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103


In [19]:
# MLflow client 생성
import mlflow
from mlflow.tracking import MlflowClient
mlflow_endpoint = "http://localhost:3000"
client = MlflowClient(tracking_uri=mlflow_endpoint)
# 모델 다운로드
download_path = "./"
mlflow_run_id = "e245a6552369498b8433e95a6ab9017c"
mlflow_run_id_artifacts_name = "keras_seg2_model_non_ewc"
client.download_artifacts(mlflow_run_id, mlflow_run_id_artifacts_name, dst_path=download_path)
# 다운로드 모델 load & predict 예시
reconstructed_model = mlflow.keras.load_model("{download_path}/{model_name}".format(download_path=download_path,model_name=mlflow_run_id_artifacts_name))
